In [1]:
import pandas as pd
import numpy as np
import pandas_profiling
from sklearn.preprocessing import StandardScaler
import torch
from torch import optim

In [2]:
import random

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


In [3]:
train = pd.read_csv('train.csv')
test =pd.read_csv('test.csv')

In [4]:
y_label = train.label
x = train.drop('label', axis=1)

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y_label)

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)
test = sc.transform(test)

In [7]:
label_cnt = pd.Series(y).drop_duplicates().count()

In [8]:
W = torch.zeros((x.shape[1], label_cnt), requires_grad=True)
b = torch.zeros(label_cnt, requires_grad=True)

In [9]:
x_tensor = torch.FloatTensor(x)
y_tensor = torch.LongTensor(y)
test_tensor = torch.FloatTensor(test)

In [10]:
x_tensor

tensor([[-7.7403e-01, -5.2260e-01, -6.2872e-01,  ..., -9.5126e-01,
          2.1998e+00, -6.8601e-01],
        [-8.8126e-01, -7.9377e-01, -1.3794e-01,  ...,  9.3986e-01,
          9.0203e-01,  4.1819e-02],
        [ 2.7147e-01,  4.4153e-01,  3.8739e-02,  ...,  9.0291e-01,
          4.1051e-01,  2.5427e+00],
        ...,
        [-1.0689e+00, -5.2260e-01, -3.3426e-01,  ..., -9.0536e-01,
          1.7058e-03, -4.0722e-02],
        [-1.0689e+00,  2.1288e+00,  3.0620e+00,  ...,  3.8248e-01,
         -1.4534e-01, -5.6274e-01],
        [-5.0595e-01,  7.4282e-01, -5.1094e-01,  ..., -4.2386e-01,
          6.8552e-01, -8.8819e-01]])

In [11]:
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 10000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.nn.functional.softmax(x_tensor.matmul(W) + b, dim=1) # or .mm or @


    ## 중간 생략 

    cost = torch.nn.functional.cross_entropy((x_tensor.matmul(W) + b), y_tensor)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))


Epoch    0/10000 Cost: 3.091042
Epoch  100/10000 Cost: 0.584040
Epoch  200/10000 Cost: 0.403136
Epoch  300/10000 Cost: 0.321740
Epoch  400/10000 Cost: 0.273493
Epoch  500/10000 Cost: 0.241013
Epoch  600/10000 Cost: 0.217402
Epoch  700/10000 Cost: 0.199319
Epoch  800/10000 Cost: 0.184939
Epoch  900/10000 Cost: 0.173170
Epoch 1000/10000 Cost: 0.163322
Epoch 1100/10000 Cost: 0.154932
Epoch 1200/10000 Cost: 0.147677
Epoch 1300/10000 Cost: 0.141328
Epoch 1400/10000 Cost: 0.135713
Epoch 1500/10000 Cost: 0.130704
Epoch 1600/10000 Cost: 0.126200
Epoch 1700/10000 Cost: 0.122124
Epoch 1800/10000 Cost: 0.118413
Epoch 1900/10000 Cost: 0.115016
Epoch 2000/10000 Cost: 0.111893
Epoch 2100/10000 Cost: 0.109009
Epoch 2200/10000 Cost: 0.106336
Epoch 2300/10000 Cost: 0.103850
Epoch 2400/10000 Cost: 0.101531
Epoch 2500/10000 Cost: 0.099361
Epoch 2600/10000 Cost: 0.097324
Epoch 2700/10000 Cost: 0.095409
Epoch 2800/10000 Cost: 0.093604
Epoch 2900/10000 Cost: 0.091900
Epoch 3000/10000 Cost: 0.090286
Epoch 31

In [15]:
hypothesis = torch.nn.functional.softmax(test_tensor.matmul(W) + b, dim=1) # or .mm or @
predict = torch.argmax(hypothesis, dim=1)

In [16]:
submit = pd.read_csv('submit.csv')
submit.label = le.inverse_transform(predict)
submit.to_csv('submit.csv', index=False)
